In [1]:
import pandas as pd
from pyspark.sql import SparkSession
import pyspark
from pyspark.sql.functions import col, countDistinct, when, broadcast, count, row_number, substring,udf,dense_rank,format_string,first,lit, to_date, transform,collect_set
from pyspark.sql.window import Window
import unicodedata
from pyspark.sql.types import StringType
import numpy as np
import pyspark.sql.functions as F


# Sparkセッションがまだ存在しない場合に作成
spark = SparkSession.builder \
    .appName("LoadParquetFile") \
    .config("spark.executor.memory", "32g") \
    .config("spark.driver.memory", "16g") \
    .config("spark.sql.shuffle.partitions", "500") \
    .config("spark.sql.files.maxPartitionBytes", "32m") \
    .getOrCreate()

file_path = "/Users/takami.soshi/Documents/analytics/chart_disease_merge/final_chart_patient_disease_with_icd.parquet"
df = spark.read.parquet(file_path)

soap_path = "/Users/takami.soshi/Documents/analytics/SOAP/preprocessed_soap"
soap_df = spark.read.parquet(soap_path)

df_lab = spark.read.parquet("/Users/takami.soshi/Documents/analytics/data2/data_for_model_with_dz_ord_by_chart_and_ordDisease.parquet")

ch_pat_path = "/Users/takami.soshi/Documents/analytics/row/chart_patient.csv"
ch_pat_df = spark.read.option("header",True).csv(ch_pat_path)


order_path = "/Users/takami.soshi/Documents/analytics/order_data_processed/grouped_recipe_shinsatsu_filtered.parquet"
order_df = spark.read.parquet(order_path)
# order_dfのshinsatsuが1のものだけ抽出
order_df = order_df.filter(F.col("shinsatsu") == 1) 
order_df = order_df.drop("create_datetime")

25/11/04 18:26:46 WARN Utils: Your hostname, Donuts-NM3044-2508.local resolves to a loopback address: 127.0.0.1; using 10.10.35.188 instead (on interface en0)
25/11/04 18:26:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/04 18:26:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/04 18:26:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## まーじ

In [ ]:
from pyspark.sql import functions as F

# === 1) order_df のカラムに "_ord" サフィックスを付ける（chart_idは除外） ===
order_df_renamed = order_df.select(
    *[
        F.col(c).alias(f"{c}_ord") if c != "chart_id" else F.col("chart_id")
        for c in order_df.columns
    ]
)

# === 2) order_df を基準に df を結合（order_df LEFT JOIN df） ===
merged_df = (
    order_df_renamed
      .join(df, on="chart_id", how="left")  # ← order_df の行をすべて保持
)

# === 3) 結果の確認 ===
merged_df.show(truncate=False)
print("結合後の件数:", merged_df.count())
print("結合後のカラム数:", len(merged_df.columns))


ERROR:root:KeyboardInterrupt while sending command.               (1 + 10) / 16]
Traceback (most recent call last):
  File "/Users/takami.soshi/Documents/analytics/my_spark_env/lib/python3.13/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/takami.soshi/Documents/analytics/my_spark_env/lib/python3.13/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ~~~~~~~~~~~~~~~~~~~~^^
  File "/opt/homebrew/Cellar/python@3.13/3.13.5/Frameworks/Python.framework/Versions/3.13/lib/python3.13/socket.py", line 719, in readinto
    return self._sock.recv_into(b)
           ~~~~~~~~~~~~~~~~~~~~^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:

ch_pat_df = ch_pat_df.select("chart_id", "clinic_id", "patient_id", "department_name", "exam_datetime", "full_name", "gender_type", "birth_date", "pregnant_class")


# === 1) ch_pat_df のカラムに "_ord" サフィックスを付ける（chart_idは除外） ===
ch_pat_df_renamed = ch_pat_df.select(
    *[
        F.col(c).alias(f"{c}_ch") if c != "chart_id" else F.col("chart_id")
        for c in ch_pat_df.columns
    ]
)

# === 2) ch_pat_df を基準に df を結合（ch_pat_df LEFT JOIN df） ===
merged_df = (
    merged_df
      .join(ch_pat_df_renamed, on="chart_id", how="left")  # ← ch_pat_df の行をすべて保持
)

# # === 3) 結果の確認 ===
merged_df.show(truncate=False)
print("結合後の件数:", merged_df.count())
print("結合後のカラム数:", len(merged_df.columns))


25/10/15 15:34:47 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+-------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

結合後の件数: 2939919
結合後のカラム数: 31


In [ ]:

soap_df = soap_df.drop("create_datetime")
# === 2) order_df を基準に df を結合（order_df LEFT JOIN df） ===
merged_df = (
    merged_df
      .join(soap_df, on="chart_id", how="left")  # ← order_df の行をすべて保持
)

merged_df.show()
print("結合後の件数:", merged_df.count())
print("結合後のカラム数:", len(merged_df.columns))


+--------+-------------+--------------------+------------------------------------+--------------------+--------------------+--------------------+----------------------------+--------------------+----------------------------------+--------------------+----------------------------------+--------------------+----------+-------------+----------------------------------+------------------+--------------------------------+--------------------+------------------------------+--------------+----------------------------------+--------------------+------------+-------------+------------------+--------------------+----------------------+--------------+--------------------+-----------------+-------------------------------------+
|chart_id|shinsatsu_ord|        category_ord|                            name_ord|           srycd_ord|       recipe_id_ord|        quantity_ord|                    unit_ord|          ykzkbn_ord|                     test_name_ord|       jlac_code_ord|            medical_clas

結合後の件数: 2939919
結合後のカラム数: 32


In [ ]:
merged_df = merged_df.drop("patient_id")

In [ ]:
merged_df.write.parquet("/Users/takami.soshi/Documents/analytics/project/shosin_disease_pred/shoshin_ch_ord_dis_soap.parquet", mode="overwrite")

## 前処理

In [2]:
df  = spark.read.parquet("/Users/takami.soshi/Documents/analytics/project/shosin_disease_pred/shoshin_ch_ord_dis_soap.parquet")
# df= df.select("chart_id", "main_name_list", "suspect_name_list", "acute_name_list", "treat_name_list", "soap_text")


In [5]:
from pyspark.sql import functions as F, types as T



mecab_dic = "/opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd"

@F.udf(T.ArrayType(T.StringType()))
def mecab_tokenize_udf(text):
    if text is None:
        return []
    try:
        # 各ワーカーで一度だけ Tagger を初期化（lazy singleton）
        import MeCab
        global _mecab_tagger
        try:
            tagger = _mecab_tagger
        except NameError:
            _mecab_tagger = MeCab.Tagger(f"-Owakati -d {mecab_dic}")
            tagger = _mecab_tagger

        parsed = tagger.parse(text)
        if not parsed:
            return []
        return [t for t in parsed.strip().split() if t]
    except Exception:
        return []

df_tok = df.withColumn("soap_tokens", mecab_tokenize_udf(F.col("soap_text")))

# 確認
df_tok.select("soap_text", "soap_tokens").show(10, truncate=False)
# df_tok.write.parquet("/Users/takami.soshi/Documents/analytics/project/shosin_disease_pred/shoshin_ch_ord_dis_soap_tok.parquet", mode="overwrite")


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

25/11/04 17:40:12 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [ ]:
from pyspark.sql import functions as F, types as T
theseurus_path ="/Users/takami.soshi/Documents/analytics/KG/医療シソーラス/dis_thesaurus_with_icd.parquet"
df_thesaurus = spark.read.parquet(theseurus_path)

node_df = spark.read.parquet("/Users/takami.soshi/Documents/analytics/KG/cytoscape/nodes_v3.parquet")
df_tok = spark.read.parquet("/Users/takami.soshi/Documents/analytics/project/shosin_disease_pred/shoshin_ch_ord_dis_soap_tok.parquet")  
# === thesaurus の全同義語をリスト化 ===
DELIM = r'[,\|/;；、，\s]+'

@F.udf(T.ArrayType(T.StringType()))
def split_to_list_udf(x):
    import re
    if x is None:
        return []
    if isinstance(x, str):
        return [p.strip() for p in re.split(DELIM, x) if p and p.strip()]
    if isinstance(x, (list, tuple)):
        return [str(v).strip() for v in x if v is not None and str(v).strip()]
    return []

# thesaurus 側の同義語を explode
th = (
    df_thesaurus
      .withColumn("synonym_list", split_to_list_udf(F.col("同義語統合リスト")))
      .select(F.explode("synonym_list").alias("synonym"))
      .filter(F.col("synonym").isNotNull() & (F.col("synonym") != ""))
      .distinct()
)

# === 全同義語を Python set に収集（小さい辞書ならOK） ===
synonym_set = set(th.toPandas()["synonym"].tolist())
broadcast_synonyms = spark.sparkContext.broadcast(synonym_set)

# === UDF: 各 soap_tokens に対してマッチしたトークンを返す ===
@F.udf(T.ArrayType(T.StringType()))
def match_tokens_udf(tokens):
    if not isinstance(tokens, list):
        return []
    syns = broadcast_synonyms.value
    return [t for t in tokens if t in syns]

# === 適用 ===
df_matched = df_tok.withColumn("matched_tokens", match_tokens_udf(F.col("soap_tokens")))

# === 確認 ===
df_matched.select("soap_text", "matched_tokens").show(10)
df_matched.write.parquet("/Users/takami.soshi/Documents/analytics/project/shosin_disease_pred/shoshin_ch_ord_dis_soap_matched.parquet", mode="overwrite")


+-------------------------------------+------------------------------+
|                            soap_text|                matched_tokens|
+-------------------------------------+------------------------------+
|                                 NULL|                            []|
|      物理療法実施\n\n処方希望\n変...|                            []|
|・ここ数日で発熱はなし　胃腸の調子...|[発熱, 便秘, 無呼吸, 腹部膨...|
|【主訴】倦怠感、背部痛、下痢【現病...| [背部痛, 下痢, 下痢, よう,...|
|      \n### 症状を教えてください（...|  [症状, 発熱, 咳, 痛み, 頭...|
|    7/27に外出して熱中症気味になっ...|[熱中症, 腹痛, 下痢, 水様便...|
|           【診察時間】 07/31 12:5...|        [体重, COVID19, COV...|
|   S) ＞皮膚にできものができて急に...|[眠気, 頭痛, 吐き気, かゆみ...|
|   S)　2日前から咽頭痛、咳、鼻水が...|          [咳, 筋肉痛, 関節痛]|
|                2024/08/01 (木) 10...|        [体重, よう, 皮膚乾燥]|
+-------------------------------------+------------------------------+
only showing top 10 rows



25/11/04 17:11:58 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:
from pyspark.sql import functions as F, types as T
df_matched = spark.read.parquet("/Users/takami.soshi/Documents/analytics/project/shosin_disease_pred/shoshin_ch_ord_dis_soap_tok.parquet")
# === thesaurus: 同義語ごとに icd10_2013 を展開 ===
DELIM = r'[,\|/;；、，\s]+'

@F.udf(T.ArrayType(T.StringType()))
def split_to_list_udf(x):
    import re
    if x is None:
        return []
    if isinstance(x, str):
        return [p.strip() for p in re.split(DELIM, x) if p and p.strip()]
    if isinstance(x, (list, tuple)):
        return [str(v).strip() for v in x if v is not None and str(v).strip()]
    return []

# 同義語をexplodeして、各語にICDを紐づける
th = (
    df_thesaurus
      .withColumn("synonym_list", split_to_list_udf(F.col("同義語統合リスト")))
      .select(F.explode("synonym_list").alias("synonym"), "icd10_2013")
      .filter(F.col("synonym").isNotNull() & (F.col("synonym") != ""))
)

# 重複除去（同義語が複数ICD持っていたら最初のを採用）
th = th.dropDuplicates(["synonym"])

# === Python dict に変換して broadcast ===
syn_to_icd = {r["synonym"]: r["icd10_2013"] for r in th.toPandas().to_dict(orient="records")}
broadcast_dict = spark.sparkContext.broadcast(syn_to_icd)

# === UDF: トークンごとにICDを付与 ===
@F.udf(T.ArrayType(
    T.StructType([
        T.StructField("token", T.StringType(), True),
        T.StructField("icd10_2013", T.StringType(), True)
    ])
))
def match_token_icd_udf(tokens):
    if not isinstance(tokens, list):
        return []
    syn_dict = broadcast_dict.value
    out = []
    for t in tokens:
        if t in syn_dict:
            out.append({"token": t, "icd10_2013": syn_dict[t]})
    return out

# === 適用 ===
df_with_icd = df_tok.withColumn("matched_token_icd", match_token_icd_udf(F.col("soap_tokens")))

# === 確認 ===
df_with_icd.show(10)
df_with_icd.write.parquet("/Users/takami.soshi/Documents/analytics/project/shosin_disease_pred/shoshin_ch_ord_dis_soap_matched_icd.parquet", mode="overwrite")


+--------+-------------+--------------------+------------------------------------+--------------------+--------------------+--------------------+----------------------------+--------------------+--------------------+--------------------+----------------------------------+--------------------+-------------+------------------------+-------------+--------------------------------+--------------------+---------------+--------------+-----------------------------------+--------------------+------------+-------------+------------------------+--------------------+----------------+--------------+--------------------+-----------------+-------------------------------------+------------------------------+-------------------------+
|chart_id|shinsatsu_ord|        category_ord|                            name_ord|           srycd_ord|       recipe_id_ord|        quantity_ord|                    unit_ord|          ykzkbn_ord|       test_name_ord|       jlac_code_ord|            medical_class_name_ord|

In [3]:
from pyspark.sql import functions as F
df_with_icd = spark.read.parquet("/Users/takami.soshi/Documents/analytics/project/shosin_disease_pred/shoshin_ch_ord_dis_soap_matched_icd.parquet")
# === 1&2: 重複除去 + 「よう」を含むトークンを除外 ===
df_filtered = df_with_icd.withColumn(
    "matched_token_icd_clean",
    F.expr("""
        filter(
            array_distinct(matched_token_icd),
            x -> NOT contains(x.token, 'よう')
        )
    """)
)

# === 3: 空配列([])の行を削除 ===
df_final = df_filtered.filter(F.size(F.col("matched_token_icd_clean")) > 0)
df_final = df_final.drop("soap_tokens", "matched_token_icd")
# === 確認 ===
df_final.select(
    "soap_text",
    "matched_token_icd_clean"
).show(10, truncate=False)
# df_final.write.parquet("/Users/takami.soshi/Documents/analytics/project/shosin_disease_pred/shoshin_ch_ord_dis_soap_clean_matched_icd.parquet", mode="overwrite")


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## プレデータ

In [4]:
import json

with open("/Users/takami.soshi/Documents/analytics/KG/医療シソーラス/synonym_to_icd.json", "r", encoding="utf-8") as f:
    synonym_to_icd = json.load(f)

from pyspark.sql import functions as F
df = spark.read.parquet("/Users/takami.soshi/Documents/analytics/project/shosin_disease_pred/shoshin_ch_ord_dis_soap_clean_matched_icd.parquet")

df_filtered = df.withColumn(
    "matched_token_icd_clean",
    F.expr("""
        filter(
            matched_token_icd_clean,
            x -> x.icd10_2013 IS NOT NULL AND x.icd10_2013 <> ''
        )
    """)
)
# df_filtered.select("main_name_list","suspect_name_list","acute_name_list","treat_name_list","main_icd_list","suspect_icd_list","acute_icd_list","treat_icd_list","matched_token_icd_clean","soap_text").write.parquet("/Users/takami.soshi/Documents/analytics/project/shosin_disease_pred/tmp.parquet", mode="overwrite")
df_filtered = df_filtered.select("chart_id","main_name_list","suspect_name_list","acute_name_list","matched_token_icd_clean")
triples = spark.read.csv("/Users/takami.soshi/Documents/analytics/KG/cytoscape/merged_triples_all.csv", header=True, inferSchema=True)


In [5]:
df.show()

+--------+-------------+--------------------+------------------------------------+--------------------+--------------------+--------------------+----------------------------+--------------------+----------------------------------+--------------------+----------------------------------+--------------------+-------------+------------------+-------------+--------------------------------+--------------------+---------------+--------------+-----------------------------------+--------------------+------------+-------------+------------------------+--------------------+----------------+--------------+--------------------+-----------------+-------------------------------------+-------------------------+
|chart_id|shinsatsu_ord|        category_ord|                            name_ord|           srycd_ord|       recipe_id_ord|        quantity_ord|                    unit_ord|          ykzkbn_ord|                     test_name_ord|       jlac_code_ord|            medical_class_name_ord| second_q

25/11/02 10:00:29 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [12]:
from pyspark.sql import functions as F

TERM = "痛風"
df.filter(F.array_contains(F.col("main_name_list"), TERM)).show(truncate=False)


+--------+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

25225719

25/11/02 17:08:49 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 996096 ms exceeds timeout 120000 ms
25/11/02 17:08:49 WARN SparkContext: Killing executors is not supported by current scheduler.
25/11/02 17:26:28 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [11]:
df.filter(F.col("chart_id")=="25225719").show(truncate=False)

+--------+-------------+------------------------------------------------------------+--------------------------------------------------------------------------------------------------+-----------------------------------------------------------------+-------------------------------------------------------+-----------------------+------------------------+-------------------------+------------------------------+------------------------------+--------------------------------------------------------------------------------------------------------------+------------------------------+-------------+----------------+-------------+-----------------+----------------+---------------+--------------+---------------+--------------+------------+-------------+------------------+-----------------------+------------+--------------+-----------------------+-----------------+-------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# === simple match → triples → NG tail 除外（重複・順序保持） ===
import pandas as pd
import numpy as np
import unicodedata
from tqdm.auto import tqdm

# 0) 正規化
def normalize_nfkc(s):
    if s is None:
        return ""
    return unicodedata.normalize("NFKC", str(s)).strip()

# 1) matched_only
matched_only = df_in[df_in["matched_tokens"].map(lambda x: isinstance(x, list) and len(x) > 0)].copy()

# 2) triples 作成（head=title, relation=h2_text, sub_relation=h3_text, tail=matched_tokensの各要素）
records = []
for _, r in matched_only.iterrows():
    head = r.get("title")
    relation = r.get("h2_text")
    sub_relation = r.get("h3_text")
    tails = r.get("matched_tokens") or []
    for t in tails:
        if t is None:
            continue
        tail = str(t)
        if tail != "":
            records.append(
                {"head": head, "relation": relation, "sub_relation": sub_relation, "tail": tail}
            )

df_triples = pd.DataFrame(records, columns=["head", "relation", "sub_relation", "tail"])

# 3) NG tail 除外（NFKC で正規化して完全一致）
NG = ["症状", "よう", "疾患", "C", "P", "G"]
NG_SET = {normalize_nfkc(s) for s in NG}

if not df_triples.empty:
    df_triples["_tail_norm"] = df_triples["tail"].map(normalize_nfkc)
    df_triples = (
        df_triples[
            (df_triples["_tail_norm"] != "") & (~df_triples["_tail_norm"].isin(NG_SET))
        ]
        .drop(columns="_tail_norm")
        .reset_index(drop=True)
    )

# （任意）完全重複の除去
# df_triples = df_triples.drop_duplicates().reset_index(drop=True)

# （任意）出現回数の付与
# df_triples_count = (
#     df_triples.groupby(["head", "relation", "sub_relation", "tail"])
#     .size().reset_index(name="count")
# )


NameError: name 'node_df' is not defined

In [ ]:
# === simple match → triples → NG tail 除外（重複・順序保持） ===
import pandas as pd
import numpy as np
import unicodedata
from tqdm.auto import tqdm

# 0) 正規化
def normalize_nfkc(s):
    if s is None:
        return ""
    return unicodedata.normalize("NFKC", str(s)).strip()

# 1) matched_only
matched_only = df_in[df_in["matched_tokens"].map(lambda x: isinstance(x, list) and len(x) > 0)].copy()

# 2) triples 作成（head=title, relation=h2_text, sub_relation=h3_text, tail=matched_tokensの各要素）
records = []
for _, r in matched_only.iterrows():
    head = r.get("title")
    relation = r.get("h2_text")
    sub_relation = r.get("h3_text")
    tails = r.get("matched_tokens") or []
    for t in tails:
        if t is None:
            continue
        tail = str(t)
        if tail != "":
            records.append(
                {"head": head, "relation": relation, "sub_relation": sub_relation, "tail": tail}
            )

df_triples = pd.DataFrame(records, columns=["head", "relation", "sub_relation", "tail"])

# 3) NG tail 除外（NFKC で正規化して完全一致）
NG = ["症状", "よう", "疾患", "C", "P", "G"]
NG_SET = {normalize_nfkc(s) for s in NG}

if not df_triples.empty:
    df_triples["_tail_norm"] = df_triples["tail"].map(normalize_nfkc)
    df_triples = (
        df_triples[
            (df_triples["_tail_norm"] != "") & (~df_triples["_tail_norm"].isin(NG_SET))
        ]
        .drop(columns="_tail_norm")
        .reset_index(drop=True)
    )

# （任意）完全重複の除去
# df_triples = df_triples.drop_duplicates().reset_index(drop=True)

# （任意）出現回数の付与
# df_triples_count = (
#     df_triples.groupby(["head", "relation", "sub_relation", "tail"])
#     .size().reset_index(name="count")
# )


NameError: name 'node_df' is not defined

In [2]:
df.select("chart_id","clinic_id_ch","department_name_ch","soap_text","matched_token_icd_clean").write.mode("overwrite").parquet("/Users/takami.soshi/Documents/analytics/project/shosin_disease_pred/for_model/soap_with_icd.parquet")

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `clinic_id_ch` cannot be resolved. Did you mean one of the following? [`chart_id`, `main_icd_list`, `patient_id`, `acute_icd_list`, `treat_icd_list`].;
'Project [chart_id#1, 'clinic_id_ch, 'department_name_ch, 'soap_text, 'matched_token_icd_clean]
+- Relation [patient_id#0,chart_id#1,exam_datetime#2,main_name_list#3,main_icd_list#4,suspect_name_list#5,suspect_icd_list#6,acute_name_list#7,acute_icd_list#8,treat_name_list#9,treat_icd_list#10] parquet


In [12]:
from pyspark.sql import functions as F, Window
NEIGHBOR_PATH   = "/Users/takami.soshi/Documents/analytics/project/shosin_disease_pred/undirected_symptom.parquet"   # あなたの保存先に合わせて

# === パラメータ（例） ===
TOP_K = 100
DEGREE_PENALTY = 0.3
DEGREE_SUPPORT_MAX = 200   # support元（= SOAPトークン）として許容する最大次数（要調整）
STOP_SYMPTOMS = ["発熱", "頭痛", "咳", "倦怠感"]  # 明示的に除外したい汎用表現を列挙（適宜）

CHART_LIMIT     = 10_000
TOP_K           = 100
DEGREE_PENALTY  = 0.3

# 近傍インデックス: node, neighbors, degree （既に読み込み済み想定）
# neighbors_idx = spark.read.parquet(NEIGHBOR_PATH).select("node","neighbors","degree")

from pyspark.sql import functions as F
from pyspark.sql.types import StringType

synonym_to_icd_bc = spark.sparkContext.broadcast(synonym_to_icd)

@F.udf(StringType())
def normalize_name(name):
    if name is None:
        return None
    return synonym_to_icd_bc.value.get(name, name)


# ===== 0) main / suspect / acute のどれかに病名がある chart_id のみ抽出 =====
valid_charts = (
    df
      .filter(
          (F.size(F.coalesce(F.col("main_name_list"), F.array().cast("array<string>"))) > 0) |
          (F.size(F.coalesce(F.col("suspect_name_list"), F.array().cast("array<string>"))) > 0) |
          (F.size(F.coalesce(F.col("acute_name_list"), F.array().cast("array<string>"))) > 0)
      )
      .select("chart_id").distinct()
)

# ===== 1) その中から上限 10,000 件を抽出 =====
charts_10k = (
    valid_charts
      .orderBy("chart_id")              # ランダムにしたい場合は .orderBy(F.rand())
      .limit(CHART_LIMIT)
)

df_10k = df.join(charts_10k, "chart_id", "inner")

# ===== 2) KGの無向1-hopインデックスを読み込み =====
neighbors_idx = (
    spark.read.parquet(NEIGHBOR_PATH)
         .select("node",
                 F.array_sort(F.array_distinct(F.col("neighbors"))).alias("neighbors"),
                 F.col("degree").cast("int").alias("degree"))
)

chart_names = (
    df_10k
      .select("chart_id", F.explode_outer("matched_token_icd_clean").alias("x"))
      .select("chart_id", normalize_name(F.col("x.token")).alias("name"))
      .filter(F.col("name").isNotNull() & (F.col("name") != ""))
      .dropDuplicates()
)

chart_name_set = (
    chart_names.groupBy("chart_id")
               .agg(
                   F.collect_set("name").alias("chart_names"),
                   F.countDistinct("name").alias("chart_name_total")
               )
)


# === 【変更点】汎用症状を“支持元”から除外して候補展開 ===
cand_raw = (
    chart_names
      .join(F.broadcast(neighbors_idx), chart_names.name == neighbors_idx.node, "inner")
      # support元の次数を保持
      .withColumn("support_degree", F.col("degree"))
      # 1) ストップリスト除外
      .filter(~F.col("name").isin(STOP_SYMPTOMS))
      # 2) support元の次数が大きすぎるものを除外
      .filter(F.col("support_degree") <= F.lit(DEGREE_SUPPORT_MAX))
      # 近傍を候補へ展開
      .select(
          chart_names.chart_id,
          chart_names.name.alias("support_name"),
          F.explode_outer("neighbors").alias("candidate")
      )
      .filter(F.col("candidate").isNotNull() & (F.col("candidate") != ""))
      .dropDuplicates()
)

# （参考）自己ヒットはそのまま保持：汎用症状の除外は “neighbors 展開” にだけ適用
self_raw = (
    chart_names
      .select("chart_id",
              F.col("name").alias("candidate"),
              F.col("name").alias("support_name"))
      .dropDuplicates()
)

cand_all = cand_raw.unionByName(self_raw).dropDuplicates()

# === 集計 → 特徴量 → スコア → Top-K（前回と同じ） ===
cand_agg = (
    cand_all.groupBy("chart_id","candidate")
            .agg(F.collect_set("support_name").alias("support_names"),
                 F.countDistinct("support_name").alias("support_count"))
)

feat_base = (
    cand_agg
      .join(chart_name_set, "chart_id", "left")
      .withColumn("self_hit", F.expr("array_contains(chart_names, candidate)").cast("int"))
)

feat = (
    feat_base
      .join(neighbors_idx.withColumnRenamed("node","candidate"), "candidate", "left")
      .withColumn("neighbors", F.coalesce(F.col("neighbors"), F.array().cast("array<string>")))
      .withColumn("degree",    F.coalesce(F.col("degree"), F.lit(0)))
      .withColumn("overlap",   F.size(F.array_intersect(F.col("neighbors"), F.col("chart_names"))))
      .withColumn(
          "jaccard",
          F.when((F.size(F.col("neighbors")) + F.col("chart_name_total") - F.col("overlap")) > 0,
                 F.col("overlap") /
                 (F.size(F.col("neighbors")) + F.col("chart_name_total") - F.col("overlap")))
           .otherwise(F.lit(0.0))
      )
)

candidates_scored = (
    feat.withColumn(
        "simple_score",
        F.col("support_count") * F.lit(2.0) +
        F.col("self_hit")      * F.lit(1.5) +
        F.col("jaccard")       * F.lit(1.0) -
        F.log1p(F.col("degree")) * F.lit(DEGREE_PENALTY)
    )
)

w = Window.partitionBy("chart_id").orderBy(
    F.desc("simple_score"),
    F.desc("support_count"),
    F.desc("jaccard")
)
candidates_topK = (
    candidates_scored
      .withColumn("rank", F.row_number().over(w))
      .filter(F.col("rank") <= F.lit(TOP_K))
      .drop("rank")
)


# ===== 7) 教師ラベル付与：主・疑い・急性 =====
labels = df_10k.select(
    "chart_id",
    F.coalesce(F.col("main_name_list"),    F.array().cast("array<string>")).alias("main_names"),
    F.coalesce(F.col("suspect_name_list"), F.array().cast("array<string>")).alias("suspect_names"),
    F.coalesce(F.col("acute_name_list"),   F.array().cast("array<string>")).alias("acute_names"),
)

training_ds = (
    candidates_topK
      .join(labels, "chart_id", "left")
      .withColumn("is_main",    F.expr("array_contains(main_names,    candidate)").cast("int"))
      .withColumn("is_suspect", F.expr("array_contains(suspect_names, candidate)").cast("int"))
      .withColumn("is_acute",   F.expr("array_contains(acute_names,   candidate)").cast("int"))
      .drop("main_names","suspect_names","acute_names")
)

# ===== 8) 結果の確認 =====
training_ds.select("chart_id", "candidate", "support_count", "simple_score", "is_main", "is_suspect", "is_acute").show(10, truncate=False)
print("chart数:", training_ds.select("chart_id").distinct().count())
print("候補行数:", training_ds.count())




+--------+--------------+-------------+------------------+-------+----------+--------+
|chart_id|candidate     |support_count|simple_score      |is_main|is_suspect|is_acute|
+--------+--------------+-------------+------------------+-------+----------+--------+
|22417426|かぜ          |1            |3.5               |0      |0         |0       |
|22417426|徴候と症状    |1            |3.5               |0      |0         |0       |
|22417426|橋本病        |1            |2.7305151927615388|0      |0         |0       |
|22417426|倦怠感・だるさ|2            |2.6756313113025803|0      |0         |0       |
|22417426|鉄欠乏性貧血  |1            |2.601280317933803 |0      |0         |0       |
|22417426|疲労          |2            |2.5294565399288036|0      |0         |0       |
|22417426|鉄欠乏症      |1            |2.0420558458320164|0      |0         |0       |
|22417426|異食症        |1            |2.0420558458320164|0      |0         |0       |
|22417426|甲状腺腫      |1            |1.6600257691269127|0      |0         |0 

chart数: 10000


候補行数: 584358


In [13]:
training_ds = training_ds.filter(F.col("degree") <= 80)

In [ ]:
training_ds.write.parquet("/Users/takami.soshi/Documents/analytics/project/shosin_disease_pred/for_model/training_dataset10000.parquet", mode="overwrite")

25/10/21 20:36:10 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 767655 ms exceeds timeout 120000 ms
25/10/21 20:36:10 WARN SparkContext: Killing executors is not supported by current scheduler.
25/10/21 20:36:12 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$